In [109]:
import torch
import jieba
import random

import numpy as np

from data import Data
from torch import nn, optim
from datetime import datetime
from torch.utils.data import DataLoader, Dataset

In [110]:
# Set random seed.
random_seed = 0

random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [111]:
device = "cpu"

if torch.cuda.is_available():
    device = "cuda:0"

print(device)

cuda:0


In [112]:
classes = {
    0: "牡羊",
    1: "金牛",
    2: "雙子",
    3: "巨蠍",
    4: "獅子",
    5: "處女",
    6: "天秤",
    7: "天蠍",
    8: "射手",
    9: "魔羯",
    10: "水瓶",
    11: "雙魚",
}

In [113]:
raw = {i: open(f"./dataset/{classes[i]}.txt", "r", encoding='UTF-8').read() for i in range(12)}
data = Data(data=raw, padding_length=32)

train_raw = data.get("data")

Cleaning completed.
ToDataset completed.
Argumantation completed.
Tokenlization completed.
Padding completed.
Token2id completed.
Process completed.


In [114]:
len(train_raw)

5627

In [115]:
train_raw[0]

[[4048,
  1507,
  2980,
  3759,
  1681,
  3147,
  3531,
  3837,
  611,
  2004,
  2517,
  2517,
  2517,
  2517,
  2517,
  2517,
  2517,
  2517,
  2517,
  2517,
  2517,
  2517,
  2517,
  2517,
  2517,
  2517,
  2517,
  2517,
  2517,
  2517,
  2517,
  2517],
 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]

In [116]:
class CreateDataset(Dataset):
    def __init__(self, data: list, label: list):
        self.data = data
        self.label = label

    def __getitem__(self, index):
        return torch.tensor(self.data[index]), torch.tensor(self.label[index], dtype=torch.float)

    def __len__(self):
        return len(self.label)

In [117]:
d, l = [], []

for i, j in train_raw:
    d.append(i); l.append(j)

train_ds = CreateDataset(d, l)

In [118]:
train_ds[1]

(tensor([3147, 3531, 3837,  611, 2004, 1681, 4048, 1507, 2980, 3759, 2517, 2517,
         2517, 2517, 2517, 2517, 2517, 2517, 2517, 2517, 2517, 2517, 2517, 2517,
         2517, 2517, 2517, 2517, 2517, 2517, 2517, 2517]),
 tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))

In [119]:
train_dl = DataLoader(train_ds, batch_size=8, shuffle=True)

In [122]:
class F_Attention(nn.Module):
    def __init__(self):
        super(F_Attention, self).__init__()
        self.query = nn.Linear(64, 64)
        self.key = nn.Linear(64, 64)
        self.value = nn.Linear(64, 64)

    def forward(self, x: torch.Tensor):
        query: torch.Tensor = self.query(x)
        key: torch.Tensor = self.key(x)
        value: torch.Tensor = self.value(x)

        score = torch.bmm(query, key.transpose(1, 2)) / 8
        attention = torch.softmax(score, dim=2)
        x = torch.bmm(attention, value)

        return x

In [120]:
class Attention(nn.Module):
    def __init__(self):
        super(Attention, self).__init__()
        self.linear1 = nn.Linear(256, 64)
        self.linear2 = nn.Linear(64, 1)
        self.relu = nn.ReLU(True)

    def forward(self, x: torch.Tensor):
        b = x.size(0)
        x = x.reshape(-1, 256)

        x = self.linear1(x)
        x = self.relu(x)
        x = self.linear2(x)

        x = x.reshape(b, -1)
        x = torch.softmax(x, dim=1)

        return x.unsqueeze(2)

In [121]:
class AttentionClassifier(nn.Module):
    def __init__(self):
        super(AttentionClassifier, self).__init__()
        self.attention = Attention()
        self.linear = nn.Linear(256, 12)

    def forward(self, x: torch.Tensor):
        attention = self.attention(x)

        x = (x * attention).sum(dim=1)
        x = torch.log_softmax(self.linear(x), dim=1)

        return x, attention

In [123]:
class Model(nn.Module):
    def __init__(self) -> None:
        super(Model, self).__init__()
        self.embedding = nn.Embedding(data.get("token_len"), 64)
        self.f_attention = F_Attention()
        self.lstm = nn.LSTM(
            input_size=64,
            hidden_size=256,
            num_layers=4,
            dropout=0.5,
            batch_first=True,
            bidirectional=True
        )
        self.linear = nn.Linear(16384, 12)
        self.classifier = AttentionClassifier()

    def forward(self, x: torch.Tensor):
        x = self.embedding(x)

        # x = self.f_attention(x)

        x, _ = self.lstm(x)

        # x = x.reshape(x.size(0), -1)
        # x = self.linear(x)

        x = x[:, :, :256] + x[:, :, :256]
        x, _ = self.classifier(x)

        return x

In [124]:
model = Model().to(device=device)

In [125]:
epochs = 30
optimizer = optim.SGD(model.parameters(), lr=0.1)
loss = nn.CrossEntropyLoss()

In [126]:
# Traning history data.
train_accuracy_h = []
train_loss_h = []
validate_accuracy_h = []

In [127]:
def train(epochs: int, model: nn.Module, optimizer: optim.Optimizer, loss: nn.CrossEntropyLoss, dataloader: DataLoader):
    # Set model to training mode.
    model.train()

    for epoch in range(epochs):
        train_accuracy = 0
        train_loss = 0
        train_total = 0
        train_process = 0
        train_time = datetime.now().timestamp()

        for texts, labels in dataloader:
            texts: torch.Tensor
            labels: torch.Tensor

            texts = texts.to(device)
            labels = labels.to(device)

            outputs: torch.Tensor = model(texts)
            losses: torch.Tensor = loss(outputs, labels)

            # optimizer.zero_grad()
            for param in model.parameters(): param.grad = None

            # Backpropagation.
            losses.backward()

            # Update parameters.
            optimizer.step()

            _, predict = torch.max(outputs, 1)
            train_accuracy += sum([labels[i][predict[i]] == 1 for i in range(len(predict))])
            train_loss += losses.item()
            train_total += labels.shape[0]
            train_process += 1

            print(
                f"{datetime.now().strftime('%Y/%m/%d %H:%M:%S')} "
                f"Epoch: {epoch:03d} "
                f"Time: {datetime.now().timestamp() - train_time:.2f} "
                f"Process: {train_process / len(dataloader) * 100:.2f}% "
                f"Accuracy: {train_accuracy / train_total * 100:.2f}% "
                f"Loss: {train_loss:.3f}",
                end="\r"
            )

        train_accuracy_h.append(train_accuracy / train_total * 100)
        train_loss_h.append(train_loss)

        print()

        # Early stop.
        if train_accuracy / train_total > 0.99:
            print("Early stopped.")
            break

    # Set model to evaluation mode.
    model.eval()

In [128]:
train(epochs=epochs, model=model, optimizer=optimizer, loss=loss, dataloader=train_dl)

2024/04/02 23:43:52 Epoch: 000 Time: 9.77 Process: 100.00% Accuracy: 10.06% Loss: 1748.167
2024/04/02 23:44:02 Epoch: 001 Time: 9.76 Process: 100.00% Accuracy: 10.93% Loss: 1743.748
2024/04/02 23:44:11 Epoch: 002 Time: 9.20 Process: 100.00% Accuracy: 11.48% Loss: 1740.928
2024/04/02 23:44:20 Epoch: 003 Time: 9.45 Process: 100.00% Accuracy: 11.85% Loss: 1735.239
2024/04/02 23:44:30 Epoch: 004 Time: 9.96 Process: 100.00% Accuracy: 13.29% Loss: 1719.685
2024/04/02 23:44:40 Epoch: 005 Time: 9.92 Process: 100.00% Accuracy: 17.20% Loss: 1669.804
2024/04/02 23:44:50 Epoch: 006 Time: 9.55 Process: 100.00% Accuracy: 24.22% Loss: 1527.779
2024/04/02 23:44:59 Epoch: 007 Time: 9.16 Process: 100.00% Accuracy: 32.15% Loss: 1375.015
2024/04/02 23:45:08 Epoch: 008 Time: 9.31 Process: 100.00% Accuracy: 38.71% Loss: 1262.126
2024/04/02 23:45:17 Epoch: 009 Time: 9.14 Process: 100.00% Accuracy: 44.50% Loss: 1136.231
2024/04/02 23:45:27 Epoch: 010 Time: 9.35 Process: 100.00% Accuracy: 53.87% Loss: 974.455


In [129]:
for i, j in train_dl:
    print(i)
    print(model(i.to(device)))
    print(j)
    break

tensor([[5047, 4536, 5122, 4141, 2995, 1681,  243, 1681, 2237, 3837, 4185, 2004,
         2517, 2517, 2517, 2517, 2517, 2517, 2517, 2517, 2517, 2517, 2517, 2517,
         2517, 2517, 2517, 2517, 2517, 2517, 2517, 2517],
        [5404,  904, 3815, 1804, 2364, 3837, 2204, 5343, 4001,   57, 2004, 1681,
         1266, 2277, 4539, 2517, 2517, 2517, 2517, 2517, 2517, 2517, 2517, 2517,
         2517, 2517, 2517, 2517, 2517, 2517, 2517, 2517],
        [1239, 3834, 2192, 3292, 3837, 4048, 3176, 2098, 4626, 4777, 3837,  611,
         5388, 5299, 1068, 1364, 2577, 3837,  799, 4048, 2975, 1681, 2936, 1364,
         3230, 4625, 1680, 3837, 4001, 2204, 4000, 1364],
        [3837, 2102, 2081,  782, 3839, 4111, 1364, 2142, 5436, 4750, 4536,  214,
         2995, 1364,  233, 2894, 1129, 1364, 5404, 1416,  763,  652,  436, 2004,
         1364, 2943, 4878,  857, 3067, 1364, 1672, 4441],
        [2326, 3843, 3837, 3526, 2218, 1364, 4574, 1364, 3894, 4057, 3869, 1364,
         2317,  214, 3947, 1364, 1704, 

In [130]:
testset = [
    "勇敢無畏，充滿了活力和冒險精神，他們喜歡追求挑戰，敢於冒險，常常是行動派的領導者。",
    "穩重可靠，以堅韌的意志力和耐心著稱，他們注重安全和舒適，並對物質生活有著強烈的執著。",
    "機智聰明，好奇心旺盛，喜歡交際和表達自己，具有多才多藝的特質，常常充滿了靈活的思維和活力。",
    "情感豐富，善解人意，對家庭和親密關係非常重視，他們總是充滿了溫柔和關懷，是很好的傾聽者和支持者。",
    "自信大方，追求著成為焦點的慾望，他們充滿了熱情和活力，喜歡引領和影響身邊的人，時常展現出優越感和領導能力。",
    "細心謹慎，追求完美，他們善於分析和解決問題，注重細節和有組織性，常常是值得信賴的夥伴和顧問。",
    "追求和諧，優雅而公正，他們注重平衡和公平，善於溝通協調，是很好的調解者和中介者。",
    "神秘內斂，充滿了熱情和直覺，他們擁有強烈的意志力和洞察力，常常是充滿挑戰性和魅力的個體。",
    "自由奔放，熱愛冒險和探索，他們追求著廣闊的視野和新鮮的體驗，時常充滿了樂觀和幽默。",
    "勤奮負責，追求事業成功和社會地位，他們具有堅毅的意志力和耐心，常常是穩健和實際的決策者。",
    "獨立思考，充滿了理想主義和創意，他們追求著獨特的生活方式和社會價值觀，常常是前衛和不拘一格的個體。",
    "敏感善良，充滿了同情心和想像力，他們常常是理想主義者和夢想家，追求著內心的情感和精神實踐。",
    "勇敢無畏，充滿活力，喜歡追求挑戰，常常是行動派的領導者。",
    "穩重可靠，堅韌耐心，注重安全舒適，對物質生活有強烈執著。",
    "機智聰明，好奇心旺盛，善於交際表達，充滿靈活思維和活力。",
    "情感豐富，善解人意，重視家庭和親密關係，溫柔關懷，傾聽支持者。",
    "自信大方，追求成為焦點，充滿熱情活力，喜歡引領影響身邊的人。",
    "細心謹慎，追求完美，善於分析解決問題，注重細節有組織性。",
    "追求和諧，優雅公正，注重平衡公平，善於溝通協調。",
    "神秘內斂，熱情直覺，意志力洞察力強，充滿挑戰性和魅力。",
    "自由奔放，熱愛冒險探索，追求廣闊視野和新鮮體驗，樂觀幽默。",
    "勤奮負責，追求事業成功社會地位，具堅毅意志力和耐心，穩健決策者。",
    "獨立思考，理想主義創意，追求獨特生活方式和價值觀，前衛不拘一格。",
    "敏感善良，同情心想像力豐富，理想主義夢想家，追求內心情感精神實踐。",
    "勇敢果敢，充滿活力，愛冒險。",
    "穩重堅定，堅持自我價值觀。",
    "靈活機智，善於溝通交際。",
    "情感豐富，家庭意識強。",
    "自信領導，熱情奔放。",
    "細心謹慎，追求完美。",
    "追求和諧，公正公平。",
    "神秘敏感，探索深度。",
    "自由探險，樂觀向上。",
    "勤奮穩健，追求成功。",
    "獨立創新，理想主義者。",
    "敏感浪漫，夢想家。",
    "生命充滿了勇氣與活力，他們勇於面對挑戰，永不退縮，常常是行動派的領袖，樂於率先嘗試新事物。",
    "展現出穩重且堅定的品性，他們始終堅持自身的價值觀與信念，喜歡在自己熟悉的領域中深耕不輟。",
    "擁有機智敏捷的頭腦，他們善於溝通交際，充滿活力與靈活的思維，對於各種新奇的事物都抱有濃厚的好奇心。",
    "的人情感豐富且懂得關懷，他們對家庭有著極為強烈的連結感，願意為了家人無條件地付出與奉獻。",
    "展現出極度的自信與熱情，他們常常是眾人注目的焦點，充滿著領導力與活力，喜歡成為團體中的中心人物。",
    "的人細心謹慎，追求完美與規律，他們擁有出色的分析能力與解決問題的技巧，喜歡保持事情的有序與井然有序。",
    "追求和諧與公正，他們擅長於溝通協調，注重平衡與公平，善於解決衝突，是團隊中不可或缺的調解者。",
    "的人神秘內斂，充滿了深度與直覺，他們擁有強烈的意志力與洞察力，總是對事情有著深入的探索與研究。",
    "熱愛自由與探險，他們樂觀向上，喜歡挑戰與冒險，追求著廣闊的視野與新鮮的體驗，不斷探索未知的領域。",
    "的人勤奮穩健，追求事業成功與社會地位，他們具有堅毅的意志力與耐心，是穩健可靠的決策者與領導者。",
    "獨立創新，理想主義者，他們勇於打破傳統的束縛，追求獨特的生活方式與社會價值觀，常常是前衛與不拘一格的個體。",
    "的人敏感浪漫，是真正的夢想家，他們充滿了同情心與想像力，常常是理想主義者，追求內心情感與精神的實踐。",
]

answare = [
    '牡羊', '金牛', '雙子', '巨蠍', '獅子', '處女', '天秤', '天蠍', '射手', '魔羯', '水瓶', '雙魚',
    '牡羊', '金牛', '雙子', '巨蠍', '獅子', '處女', '天秤', '天蠍', '射手', '魔羯', '水瓶', '雙魚',
    '牡羊', '金牛', '雙子', '巨蠍', '獅子', '處女', '天秤', '天蠍', '射手', '魔羯', '水瓶', '雙魚',
    '牡羊', '金牛', '雙子', '巨蠍', '獅子', '處女', '天秤', '天蠍', '射手', '魔羯', '水瓶', '雙魚',
]

In [131]:
def process(testset: list) -> torch.Tensor:
    result = []

    for line in testset:
        temp = jieba.lcut(line)
        temp = temp + ["<PAD>"] * (32 - len(temp))
        temp = [data.w2i[x] if x in data.w2i else data.w2i["<PAD>"] for x in temp][:32]
        result.append(temp)

    return result

In [132]:
test = process(testset)

In [133]:
model.eval()

Model(
  (embedding): Embedding(5470, 64)
  (f_attention): F_Attention(
    (query): Linear(in_features=64, out_features=64, bias=True)
    (key): Linear(in_features=64, out_features=64, bias=True)
    (value): Linear(in_features=64, out_features=64, bias=True)
  )
  (lstm): LSTM(64, 256, num_layers=4, batch_first=True, dropout=0.5, bidirectional=True)
  (linear): Linear(in_features=16384, out_features=12, bias=True)
  (classifier): AttentionClassifier(
    (attention): Attention(
      (linear1): Linear(in_features=256, out_features=64, bias=True)
      (linear2): Linear(in_features=64, out_features=1, bias=True)
      (relu): ReLU(inplace=True)
    )
    (linear): Linear(in_features=256, out_features=12, bias=True)
  )
)

In [134]:
result = model(torch.tensor(test).to(device))
print(result)

tensor([[-1.1765e-01, -1.9629e+01, -9.1400e+00, -1.5665e+01, -7.3810e+00,
         -2.0351e+01, -1.3112e+01, -8.6578e+00, -2.2074e+00, -2.0208e+01,
         -9.3177e+00, -1.1710e+01],
        [-5.3995e+00, -4.1170e+00, -3.1809e+00, -2.5395e-01, -7.5983e+00,
         -6.6918e+00, -3.8490e+00, -2.1283e+00, -4.9332e+00, -5.8055e+00,
         -5.7332e+00, -5.0565e+00],
        [-3.2537e+00, -1.0043e+01, -5.3831e+00, -3.9422e+00, -4.7011e+00,
         -7.7596e+00, -6.5669e+00, -2.2455e+00, -2.4333e-01, -1.0629e+01,
         -3.9287e+00, -4.0852e+00],
        [-1.4176e+01, -9.4901e+00, -9.7644e+00, -1.5839e-02, -1.3994e+01,
         -1.1428e+01, -5.1549e+00, -8.9585e+00, -7.2575e+00, -4.7510e+00,
         -8.0478e+00, -1.3602e+01],
        [-8.8867e-01, -1.5780e+01, -7.6809e+00, -8.8323e+00, -1.1291e+00,
         -1.3964e+01, -1.3698e+01, -1.3600e+00, -4.8254e+00, -1.4455e+01,
         -9.7673e+00, -9.0996e+00],
        [-1.8369e+01, -6.2271e+00, -1.1301e+01, -4.8079e+00, -1.2098e+01,
      

In [135]:
result = result.tolist()

In [136]:
# Normolization
results = []

for i in range(len(result)):
    temp = result[i]
    temp = [x - min(temp) for x in temp]
    temp = [x / max(temp) for x in temp]
    temp = [round(x, 3) for x in temp]

    results.append(temp)

In [137]:
count = 0

for i, r in enumerate(results):
    t = classes[int(torch.argmax(torch.tensor(r)))]
    if t == answare[i]:
        count += 1
        print(t + "✅")
    else:
        print(t + " <- " + answare[i])

print(f"Correct: {count}/{len(answare)} ({(count / len(answare)) * 100 :.2f}%)")

牡羊✅
巨蠍 <- 金牛
射手 <- 雙子
巨蠍✅
牡羊 <- 獅子
魔羯 <- 處女
天秤✅
牡羊 <- 天蠍
射手✅
射手 <- 魔羯
水瓶✅
射手 <- 雙魚
牡羊✅
魔羯 <- 金牛
射手 <- 雙子
巨蠍✅
獅子✅
處女✅
天秤✅
射手 <- 天蠍
射手✅
巨蠍 <- 魔羯
射手 <- 水瓶
射手 <- 雙魚
牡羊✅
魔羯 <- 金牛
雙子✅
天蠍 <- 巨蠍
牡羊 <- 獅子
處女✅
天秤✅
天蠍✅
射手✅
獅子 <- 魔羯
水瓶✅
天蠍 <- 雙魚
射手 <- 牡羊
射手 <- 金牛
獅子 <- 雙子
巨蠍✅
射手 <- 獅子
處女✅
天秤✅
射手 <- 天蠍
射手✅
巨蠍 <- 魔羯
水瓶✅
射手 <- 雙魚
Correct: 23/48 (47.92%)


In [138]:
import matplotlib.pyplot as plt
import numpy as np

In [139]:
# angles_A = np.linspace(start=0, stop=2*np.pi, num=len(result)+1, endpoint=True)
# values_A = np.concatenate((result, [result[0]]))

# fig, ax = plt.subplots(1, 1, figsize=(5, 5), subplot_kw={'projection': 'polar'})
# ax.plot(angles_A, values_A, 'o-', color="blue", label="A")

# ax.fill(angles_A, values_A, alpha=0.3, color="blue")
# ax.set_thetagrids(angles_A[:-1] * 180 / np.pi, range(12), fontsize=15)
# ax.set_theta_zero_location('N')

In [140]:
# Please check the virtual input of the ONNX model.
torch.onnx.export(model, torch.tensor([test[0]]).to(device=device), 'constellator.onnx')

In [141]:
import onnx
import onnx.helper

onnx_model = onnx.load('./constellator.onnx')
print(onnx.helper.printable_graph(onnx_model.graph))

graph torch_jit (
  %input.1[INT64, 1x32]
) initializers (
  %embedding.weight[FLOAT, 5470x64]
  %classifier.attention.linear1.weight[FLOAT, 64x256]
  %classifier.attention.linear1.bias[FLOAT, 64]
  %classifier.attention.linear2.weight[FLOAT, 1x64]
  %classifier.attention.linear2.bias[FLOAT, 1]
  %classifier.linear.weight[FLOAT, 12x256]
  %classifier.linear.bias[FLOAT, 12]
  %onnx::LSTM_748[FLOAT, 2x2048]
  %onnx::LSTM_749[FLOAT, 2x1024x64]
  %onnx::LSTM_750[FLOAT, 2x1024x256]
  %onnx::LSTM_795[FLOAT, 2x2048]
  %onnx::LSTM_796[FLOAT, 2x1024x512]
  %onnx::LSTM_797[FLOAT, 2x1024x256]
  %onnx::LSTM_842[FLOAT, 2x2048]
  %onnx::LSTM_843[FLOAT, 2x1024x512]
  %onnx::LSTM_844[FLOAT, 2x1024x256]
  %onnx::LSTM_889[FLOAT, 2x2048]
  %onnx::LSTM_890[FLOAT, 2x1024x512]
  %onnx::LSTM_891[FLOAT, 2x1024x256]
) {
  %/embedding/Gather_output_0 = Gather(%embedding.weight, %input.1)
  %/lstm/Transpose_output_0 = Transpose[perm = [1, 0, 2]](%/embedding/Gather_output_0)
  %/lstm/Constant_output_0 = Constant[

In [142]:
import os
print(f"ONNX model size: {os.path.getsize('./constellator.onnx') / 1048576:.2f} MB")

ONNX model size: 22.00 MB
